In [24]:
import os
import pandas as pd
from torchvision.io import read_image
import os
import json
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2 as cv
import glob
import torch
from lxml import etree
from torch import nn
import tqdm
import numpy as np
class CustomImageDataset( ):
    def __init__(self, annotations_file="E:\Content and Style Aware Generation\line_data\labels", img_dir="E:\Content and Style Aware Generation\line_data\Images", transform=None, target_transform=None):
        self.label_dict={}
        self.IMAGE_HEIGHT=10
        self.IMAGE_WIDTH=20
        self.img_path = img_dir
        #self.transform = get_transform(grayscale=True)
        self.num_example=10
        self.label_path=annotations_file
        self.label_dir = os.listdir(self.label_path)
        self.img_dir=(glob.glob(self.img_path+"/*/*/*.png"))
        del(self.label_dir[1230])
        for file in tqdm.tqdm(self.label_dir):
            tree = ET.parse(source=os.path.join(self.label_path,file), parser=None)
            root = tree.getroot()
            for child in root.iter("line"):
                data = child.attrib
                self.label_dict[data["id"]] = data["text"]
    def image_lbl(self,index):
        img=cv.imread(self.img_dir[index])
        img = 255 - img
        img_height, img_width = img.shape[0], img.shape[1]
        n_repeats = int(np.ceil(self.IMAGE_WIDTH/img_width))
        padded_image = np.concatenate([img]*n_repeats, axis=1)
        padded_image = padded_image[:self.IMAGE_HEIGHT, :self.IMAGE_WIDTH]
        resized_img = cv.resize(padded_image, (self.IMAGE_WIDTH, self.IMAGE_HEIGHT))
        label=self.label_dict[os.path.basename(self.img_dir[index])[:-4]]
        return resized_img,label
        ''' print(padded_image.shape)
        plt.imshow(padded_image)
        plt.axis('off')
        plt.show()
        print()'''
    def __len__(self):
        return len(self.label_dict)

    def __getitem__(self, idx):
        Images=list()
        Labels=list()
        data_set={}
        random_idxs = np.random.choice(len(self.img_dir), 5, replace = True)
        
        for index in random_idxs:
            Img,lbl=self.image_lbl(index)
            Images.append(torch.tensor(Img).float().to(device))
            #Images.append(torch.nn.functional.pad(tensor, (0, self.IMAGE_WIDTH - tensor.size(2), 0, self.IMAGE_HEIGHT - tensor.size(1))))
            Labels.append(lbl)
        concate_image=torch.cat(Images,0)   
        '''if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)'''
        return Images,Labels

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"

In [13]:
data=CustomImageDataset()


from torch.utils.data import DataLoader

dataset = DataLoader(data, batch_size=2, shuffle=True)


100%|████████████████████████████████████████████████████████████████████████████| 1539/1539 [00:01<00:00, 1226.80it/s]


In [15]:
class Visual_encoder(nn.Module):
    def __init__(self) -> None:
        super(Visual_encoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3, out_channels=100, kernel_size=3, stride=1, padding=1
            ),
            nn.ReLU(),
            nn.BatchNorm2d(100),
            nn.Conv2d(
                in_channels=100, out_channels=100, kernel_size=3, stride=1, padding=1
            ),
            nn.ReLU(),
        )

        self.conv2 = nn.Conv2d(
            in_channels=100, out_channels=32, kernel_size=3, stride=1, padding=1
        )
        self.conv3 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1
        )
        self.conv4 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1
        )
        self.conv5 = nn.Conv2d(
            in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1
        )
     
        self.upsample1 = nn.Upsample(scale_factor=2, mode='nearest')
        self.upsample2 = nn.Upsample(scale_factor=2, mode='nearest')
        self.upsample3 = nn.Upsample(scale_factor=2, mode='nearest')



    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x=self.upsample1(x)
        x=self.upsample2(x)
        x=self.upsample3(x)
        return x


In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdaLN(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.rho = nn.Parameter(torch.Tensor(1, num_features, 1, 1))
        self.gamma = nn.Parameter(torch.Tensor(1, num_features, 1, 1))
        self.beta = nn.Parameter(torch.Tensor(1, num_features, 1, 1))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.constant_(self.rho, 0.9)
        nn.init.constant_(self.gamma, 1.0)
        nn.init.constant_(self.beta, 0.0)

    def forward(self, x):
        mean = torch.mean(x, dim=[2, 3], keepdim=True)
        var = torch.var(x, dim=[2, 3], keepdim=True)
        x = (x - mean) / torch.sqrt(var + self.eps)
        return self.gamma * x + self.beta

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, norm_layer=nn.BatchNorm2d, activation=F.relu):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = norm_layer(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = norm_layer(out_channels)
        self.stride = stride
        self.activation = activation
        self.adaln = AdaLN(out_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation(out)
        out = self.adaln(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.activation(out)
        return out

class Generator_Resnet (nn.Module):
    def __init__(self,class_num, num_res_blocks=4, norm_layer=AdaLN, activation=F.relu):
        super().__init__()
        self.num_res_blocks = num_res_blocks
        self.norm_layer = norm_layer
        self.activation = activation

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn1 = norm_layer(64)
        self.res_blocks = nn.Sequential(*[ResidualBlock(64, 64, norm_layer=self.norm_layer, activation=self.activation) for _ in range(self.num_res_blocks)])
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.upsample2 = nn.Upsample(scale_factor=2, mode='nearest')
        
        # Second convolutional module
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.upsample3 = nn.Upsample(scale_factor=2, mode='nearest')
        
        # Third convolutional module
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.upsample4 = nn.Upsample(scale_factor=2, mode='nearest')
        
        # Fourth convolutional module
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()
        self.upsample5 = nn.Upsample(scale_factor=1, mode='nearest')
        
        # Final activation layer
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        
        x = self.res_blocks(x)
        
        x=  self.conv2(x)
        x=self.relu2(x)
        x=self.upsample2(x)
        
        x=  self.conv3(x)
        x=self.relu3(x)
        x=self.upsample3(x)
        
        x=  self.conv4(x)
        x=self.relu4(x)
        x=self.upsample4(x)
        
        x=  self.conv5(x)
        x=self.relu5(x)
        x=self.upsample5(x)
        x = self.tanh(x)
        
        return x
            




In [22]:
from torch import nn

# from torch.autograd import Variable
import numpy as np
from parameters import *
import torch.functional as F
from attention import MultiHeadAttention
# from models.vgg_tro_channel1 import vgg16_bn
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        B,C,H,W=batch_size,256,IMAGE_HEIGHT*scale_factor,IMAGE_WIDTH*scale_factor
        print(f"channel:-{C=},Hight:- {H=} width:- {W=} Batch:- {B=}")
        self.resnet=Generator_Resnet(class_num=2,num_res_blocks=6)
        self.visual_encoder=Visual_encoder()
        self.layer_norm=nn.LayerNorm([C,H,W]) #256,
        self.attention=MultiHeadAttention(num_heads=8,H=H,W=W,B=B,C=C,scale_factor=scale_factor,head_size=1000,dropout=0.4)
        self.dropout=nn.Dropout(p=0.3)
        self.linear=nn.Linear(in_features=W*H,out_features=1000,bias=False)

    def forward(self,x):
        resent=self.resnet(x)   #resent
        visual_encder=self.visual_encoder(x)  #visual encoder for positioning
        print(f"Shape of the resent output{resent.shape} and Vgg output shape{visual_encder.shape}")
        combained_out=resent+visual_encder #combained before input
        layer_norm=self.layer_norm(combained_out) #layer_norm
        attention=self.attention(layer_norm)  #attention layer
        dropout=self.dropout(attention)   #dropout layer
        norm_dropout=layer_norm+dropout   #combained output of layer_norm and dropout
        layer_norm1=self.layer_norm(norm_dropout)  #layer_norm output
        linear=self.linear(layer_norm1)     #linear layer
        linear_drop=self.dropout(linear)  
        norm_dropout1=linear_drop+norm_dropout
        final_norm=self.layer_norm(norm_dropout1)

        return final_norm




        
    




In [23]:
network=Encoder()

channel:-C=256,Hight:- H=40 width:- W=80 Batch:- B=2


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3276800000 bytes.

In [32]:
for image,label in dataset:
    
    #resnet_out=net(image[0].permute(0,3,2,1).to(device))
    print("Input Image shape",image[0].permute(0,3,2,1).shape)
    ''' print("Resnet output shape",resnet_out.shape)
    output=vvg_net(image[0].permute(0,3,2,1).to(device))
    print("Vvg network outshape",output.shape)
    combained_out=output+resnet_out
    print(combained_out.shape)
    _,C,W,H=combained_out.shape
    print("combained shape",combained_out.shape)
    layer=nn.LayerNorm([C,W,H])
    out=layer(combained_out)
    out.shape'''
    break

Input Image shape torch.Size([2, 3, 20, 10])


In [11]:
scale_factor=2**3
Hight,width=10,20


In [12]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, H,W,Scale_factor,head_size,dropout,C,B):
        super().__init__()
        self.key = nn.Linear(Scale_factor*H*W*scale_factor, head_size, bias=False)
        self.query = nn.Linear(Scale_factor*H*W*scale_factor, head_size, bias=False)
        self.value = nn.Linear(Scale_factor*H*W*scale_factor, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(B,C,C)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,H,W = x.shape 
        print("input shape",x.shape)
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        print(wei.shape)
        print(self.tril.shape)
        wei = wei.masked_fill(self.tril[:H, :H] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [19]:
attention=Head(10,20,scale_factor,1000,0.20,2,256)
out=attention(sample)


input shape torch.Size([2, 256, 12800])
torch.Size([2, 256, 256])
torch.Size([256, 2, 2])


RuntimeError: The size of tensor a (2) must match the size of tensor b (256) at non-singleton dimension 2

In [31]:
sample_tensor=torch.rand(2, 256, 160, 80)
sample=sample_tensor.view(2,256,-1)

B,C,W,H=sample_tensor.shape
l=nn.Linear(W*H,1000,bias=False)

print(W*H)

12800


In [30]:
print("Input sample",sample.shape)
out=l(sample)
print(out.shape)

drop=nn.Dropout(0.2)
out=drop(out)
print(out.view(2,256,-1))
l=nn.Linear(256,256)

Input sample torch.Size([2, 256, 12800])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x12800 and 256x256)

In [29]:
l(drop(out))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x1000 and 256x256)

In [15]:
class MultiHeadAttention(nn.Module):
  "Multiple heads of the self_attention in parallel"
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embd,n_embd)
    self.dropout=nn.Dropout(dropout)

  def forward(self,x):
    out= torch.cat([h(x) for h in self.heads],dim=-1)
    out=self.dropout (self.proj(out))
    return out

In [ ]:
print(out.view(2,-1).shape)
print(output.view(2,-1).shape)

In [ ]:
image[0].permute(0,3,2,1).shape #0,3,2,1

In [ ]:
t=image[0].view(-1,3)
t.shape

In [ ]:
result=out.permute(0,2,3,1).contiguous() #torch.Size([2, 3, 82, 402])
pred=result.view(-1,3)
pred.shape

In [ ]:
 loss = F.cross_entropy(pred,t)

In [ ]:
import torch
import torch.nn.functional as F

# Define the input tensors
input1 = torch.randn(93670, 3)  # tensor of size [93670, 3]
input2 = torch.randn(1508904, 3)  # tensor of size [1508904, 3]
print(input1.shape)
print(input2.shape)
# Define the target labels for each input tensor
target1 = torch.randint(0, 3, (93670,))  # tensor of size [93670]
print(target1.shape)
target2 = torch.randint(0, 3, (1508904//10,))  # tensor of size [1508904//10]
print
# Repeat target tensor for input tensor 2
target2 = target2.repeat(10)

# Apply cross-entropy loss to each input tensor
loss1 = F.cross_entropy(input1, target1)

print(f"Loss for tensor 1: {loss1}")


In [ ]:
#1, batch_size, self.height//16*512

In [ ]:
out.view(-1,out.size(-1)).shape

In [ ]:
out.size(-1)

In [ ]:
 loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)

In [ ]:
a=torch.rand(30030,402)

In [ ]:
import torch

# create tensors
A = torch.randn(30030, 402)
B = torch.randn(3, 2)

# transpose B
B = torch.transpose(B, 0, 1)

# perform matrix multiplication
C = torch.mm(A, B)

print(C.shape)


In [ ]:
loss = torch.nn.functional.cross_entropy(t,pred)

In [ ]:
 loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)

In [ ]:
def show_Image_logits(Image,logits):
    for img,logit in zip(Image,logits):
        plt.title("model output")
        plt.imshow(logit.permute(2,1,0).detach().numpy())
        plt.axis("off")
        plt.show()
        
        plt.title("model input")
        plt.imshow(img)
        plt.axis("off")
        plt.show()
        

In [ ]:
show_Image_logits(image[0],out)

In [43]:
import torch
import torch.nn as nn

# create a tensor of size (2, 32, 10, 20)
x = torch.randn(2, 32, 10, 20)

# flatten the spatial dimensions
x = x.view(2,-1)

# create a linear layer
linear = nn.Linear(in_features=32*10*20, out_features=100)

# apply the linear layer to the flattened tensor
output = linear(x)

# print the output size
print(output.size())  # should be (2, 32, 100)


torch.Size([2, 100])


In [40]:
20*10*32

6400

In [45]:
(torch.rand(2,1600)+torch.rand(2,1600)).shape

torch.Size([2, 1600])

In [77]:
import torch.nn as nn
import torch
import random
random.seed(200)
# Create a tensor of shape (batch_size, num_features, feature_dim1, feature_dim2)
x = torch.randn(16, 32, 10, 10)
x=x.view(16,x.size(1),-1)
# Apply layer normalization to the last two dimensions
layer_norm = nn.LayerNorm([10* 10])
x_normalized = layer_norm(x)

# Print the shapes of the original and normalized tensors
print("Original shape:", x.shape)  # (16, 32, 10, 10)
print("Normalized shape:", x_normalized.shape)  # (16, 32, 10, 10)


Original shape: torch.Size([16, 32, 100])
Normalized shape: torch.Size([16, 32, 100])


In [72]:
x[0][0][0]

tensor([-0.8666, -0.2625,  0.4433, -0.9781, -2.0363,  0.4728, -1.0781, -0.4247,
         0.9284, -0.5977])

In [69]:
x_normalized[0][0][0]

tensor([ 0.6208, -0.6617, -1.0732,  0.5020, -0.1845, -0.7720,  0.1638,  0.8357,
         1.2288,  1.0207], grad_fn=<SelectBackward0>)

In [71]:
x_normalized[0][0][0]

tensor([-0.8453, -0.2377,  0.4721, -0.9573, -2.0215,  0.5018, -1.0579, -0.4008,
         0.9599, -0.5749], grad_fn=<SelectBackward0>)

In [78]:
import torch
import torch.nn as nn

class AttentionLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AttentionLayer, self).__init__()
        self.query_layer = nn.Linear(input_dim, hidden_dim, bias=False)
        self.key_layer = nn.Linear(input_dim, hidden_dim, bias=False)
        self.scale = 1.0 / (hidden_dim ** 0.5)

    def forward(self, input):
        # Compute query and key vectors
        query = self.query_layer(input)
        key = self.key_layer(input)

        # Compute attention weights using dot product
        weights = torch.matmul(query, key.transpose(-2, -1))
        weights = weights * self.scale
        weights = nn.functional.softmax(weights, dim=-1)

        # Apply attention weights to values
        output = torch.matmul(weights, input)

        return output


In [90]:
# Instantiate attention layer with input dimension of 32 and hidden dimension of 64
attention_layer = AttentionLayer(32, 64)

# Generate a random input tensor of size [batch_size, seq_len, 32]
input_tensor = torch.randn(10, 64, 32)

# Apply attention layer to input tensor
output_tensor = attention_layer(input_tensor)


In [93]:
input_tensor[0].shape

torch.Size([64, 32])

In [83]:
1.0 / (100 ** 0.5)

0.1

In [112]:
zeros=torch.zeros((10,10))
zeros=zeros.view(-1)
zeros

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [111]:
ones=torch.ones((20,20))
ones=ones.view(-1)
ones

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [115]:
(ones.view(4,-1)+zeros).shape

torch.Size([4, 100])

In [116]:
import torch

# Create a tensor of size [2, 200]
x = torch.randn(2, 200)

# Reshape the tensor to size [2, 32, 200]
x = x.view(2, 32, 200)

# Print the new shape of the tensor
print(x.shape)  # torch.Size([2, 32, 200])


RuntimeError: shape '[2, 32, 200]' is invalid for input of size 400

In [120]:
import torch.nn as nn

# Create a linear layer with the same input and output size
linear_layer = nn.Linear(200, 200)
activation = nn.Identity()

# Create a tensor of size [2, 32, 200]
x = torch.randn(2, 32, 200)

# Apply the linear layer and activation function to the input tensor
x = activation(linear_layer(x))

# The output tensor should have the same size as the input tensor
print(x.shape)  # torch.Size([2, 32, 200])


torch.Size([2, 32, 200])


In [125]:
torch.rand(2, 12800).view(2,32,-1).shape

torch.Size([2, 32, 400])